# Lecture 6: Functions and testing

## Topic learning objectives

By the end of this topic, students should be able to:

1. Fully specify a function and write comprehensive tests against the specification.
2. Reproducibly generate test data (e.g., data frames, models, plots).
3. Discuss the observability of unit outputs in data science (e.g., plot objects), and how
this should be taken into account when designing software.
4. Explain why test-driven development (TDD) affords testability
5. Use exceptions when writing code.
6. Test if a function is throwing an exception when it should, and that is does not do so
when it shouldn’t.
7. Evaluate test suite quality.

In [1]:
# Limit output of data frame output to 10 lines
options(repr.matrix.max.rows = 10)

## Functions

- Functions are individual units of code that perform a specific task.
- They are useful for increasing code modularity - this helps with reusability and readability.
- Functions can be easily tested to ensure the correct function outputs are given and that they handle user errors in expected ways. These checks help to increase the robustness of your code.

When should you write a function? In practice, when you start re-writing code for the second or third time, it really is time to abstract your code into a function. When does this happen in data science? Think back to your DSCI 100 projects, you may have had redundant code when you:

1. Repeatedly applied some data cleaning/manipulation to columns in your raw data suring the data wrangling process

2. Created many similar data visualizations during your exploratory data analysis

3. Created many similar tidymodels workflows when you were tuning your predictive models


## Defining functions in R:

In this course, we will get to practice writing functions (and then testing them) in the R programming language. Given this has not been covered in the course pre-requisites, we will cover it here briefly. To learn more about functions in R, we refer you to the [functions chapter in the Advanced R book](https://adv-r.hadley.nz/functions.html).

- Use `variable <- function(…arguments…) { …body… }` to create a function and give it a name

Example:

In [1]:
add_two_numbers <- function(x, y) {
  x + y
}

add_two_numbers(1, 4)

[1] 5

- Functions in R are objects. This is referred to as “first-class functions”.
- The last line of the function returns the object created, or listed there. To return a value early use the special word `return` (as shown below).

In [16]:
math_two_numbers <- function(x, y, operation) {
  if (operation == "add") {
    return(x + y)
  } 
  x - y
}

In [17]:
math_two_numbers (1, 4, "add")

[1] 5

In [18]:
math_two_numbers (1, 4, "subtract")

[1] -3

### Default function arguments

Default values can be specified in the function definition:

In [21]:
math_two_numbers <- function(x, y, operation = "add") {
  if (operation == "add") {
    return(x + y)
  } 
  x - y
}

In [22]:
math_two_numbers (1, 4)

[1] 5

In [23]:
math_two_numbers (1, 4, "subtract")

[1] -3

#### Optional - Advanced

##### Extra arguments via `...`

If we want our function to be able to take extra arguments that we don't specify, we must explicitly convert `...` to a list (otherwise `...` will just return the first object passed to `...`):

In [29]:
add <- function(x, y, ...) {
    print(list(...))
}
add(1, 3, 5, 6)

[[1]]
[1] 5

[[2]]
[1] 6



### Lexical scoping in R

R’s lexical scoping follows several rules, we will cover the following 3:

- Name masking
- Dynamic lookup
- A fresh start

#### Name masking

Object names which are defined inside a function mask object names defined outside of that function. For example, below `x` is first defined in the global environment, and then also defined in the function environment. `x` in the function environment masks `x` in the global environment, so the value of 5 is used in `to_add + x` instead of 10:

In [35]:
x <- 10

add_to_x <- function(to_add) {
    x <- 5
    to_add + x
}

add_to_x(2)

[1] 7

If a function refers to an object name that is not defined inside the function, then R looks at the environment one level above. And if it is not found there, it again, looks another level above. This will happen all the way up the environment tree until it searches all possible environments for that session (including the global environment and loaded packages).

<img src="img/env-lookup.png" width=600>

*__Attribution__: Derived from [Advanced R](https://adv-r.hadley.nz/) by Hadley Wickham*

Here `x` is not defined in the function environment, and so R then looks one-level up to the global environment to try to find `x`. In this case it does and it is then used to calculate `to_add + x`:

In [38]:
x <- 10

add_to_x <- function(to_add) {
    to_add + x
}

add_to_x(2)

[1] 12

#### Dynamic lookup

R does not look up the values for objects it references when it is defined/created, instead it does this when the function is called. This can lead to the function returning different things depending on the values of the objects it references outside of the function's environment. 

For example, each time the function `add_to_x` is run, R looks up the current value of `x` and uses that to compute `to_add + x`:

In [37]:
add_to_x <- function(to_add) {
    to_add + x
}

x <- 10
add_to_x(2)

x <- 20
add_to_x(2)

[1] 12

[1] 22

#### A fresh start

Functions in R have no memory of what happened the last time they were called. This happens because a new function environment is created, R created a new environment in which to execute it. 

For example, even though we update the value of `x` inside the function by adding two to it every time we execute it, it returns the same value each time, because R creates a new function environment each time we run it and that environment has no recollection of what happened the last time the function was run:

In [41]:
x <- 10

add_to_x <- function(to_add) {
    x <- to_add + x
    x
}

add_to_x(2)
add_to_x(2)
add_to_x(2)

[1] 12

[1] 12

[1] 12

### Lazy evaluation

In R, function arguments are lazily evaluated: they’re only evaluated if accessed.



Knowing that, now consider the `add_one` function written in both R and Python below:

```
# R code (this would work)
add_one <- function(x, y) {
    x <- x + 1
    return(x)
} 
```

```
# Python code (this would not work)
def add_one(x, y):
    x = x + 1
    return x
```

Why will the above `add_one` function will work in R, but the equivalent version of the function in python would break?

- Python evaluates the function arguments before it evaluates the function and because it doesn't know what `y` is, it will break even though it is not used in the function.
- R performs lazy evaluation, meaning it delays the evaluation of the function arguments until its value is needed within/inside the function. Since `y` is never referenced inside the function, R doesn't complain, or even notice it.


#### `add_one` in R

```
add_one <- function(x, y) {
    x <- x + 1
    return(x)
} 
```

This works: 


```
add_one(2, 1)
```

```
3
```

and so does this:


```
add_one(2)
```

```
3
```

#### `add_one` in Python

```
def add_one(x, y):
    x = x + 1
    return x`
```

This works: 


```
add_one(2, 1)
```

```
3
```

This does not:


```
add_one(2)
```

```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-5-f2e542671748> in <module>
----> 1 add_one(2)

TypeError: add_one() missing 1 required positional argument: 'y'
```

#### The power of lazy evaluation

Let's you have easy to use interactive code like this:

In [44]:
dplyr::select(mtcars, mpg, cyl, hp, qsec)

,mpg,cyl,hp,qsec
,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,110,16.46
Mazda RX4 Wag,21.0,6,110,17.02
Datsun 710,22.8,4,93,18.61
Hornet 4 Drive,21.4,6,110,19.44
Hornet Sportabout,18.7,8,175,17.02
Valiant,18.1,6,105,20.22
Duster 360,14.3,8,245,15.84
Merc 240D,24.4,4,62,20.00
Merc 230,22.8,4,95,22.90


Notes: 
- There's more than just lazy evaluation happening in the code above, but lazy evaluation is part of it.
- `package::function()` is a way to use a function from an R package without loading the entire library.


## Function documentation


### Function documentation in R

In R, we write function documentation in a style that the `roxygen2` R package can use to automatically generate nicely formatted documentation, accessible via R's help function, when we create R packages. This is not available to us until we package our functions in software packages, however it is a good practice to get into because it helps ensure you document the key aspects of a function for your user, and it means that when you go to create an R package to share your function, you will have already correctly formatted the function documentation - saving future you some work!

In [23]:
#' Converts temperatures from Fahrenheit to Celsius.
#'    
#' @param temp a vector of temperatures in Fahrenheit
#' 
#' @return a vector of temperatures in Celsius
#' 
#' @examples
#' fahr_to_celcius(-20)
fahr_to_celsius <- function(temp) {
    (temp - 32) * 5/9
}

Why `roxygen2` documentation? If you document your functions like this, *when* you create an R package to share them they will be set up to have the fancy documentation that we get using `?function_name`.

## Writing tests with `testthat`

- Industry standard tool for writing tests in R is the [{testthat} package](https://testthat.r-lib.org/).
- To use an R package, we typically load the package into R using the `library` function:

In [11]:
library(testthat)

### How to write a test with `testthat`

```
test_that("Message to print if test fails", expect_*(...))
```

Often our `test_that` function calls are longer than 80 characters, so we use `{` to split the code across multiple lines, for example:

In [12]:
x <- c(3.5, 3.5, 3.5)
y <- c(3.5, 3.5, 3.5)
test_that("x and y should contain the same values", {
    expect_equal(x, y)
})

#### Common `expect_*` statements for use with `test_that`

#### Is the object equal to a value? 
- `expect_identical` - test two objects for being exactly equal
- `expect_equal` - compare R objects x and y testing ‘near equality’ (can set a tolerance)
- `expect_equivalent` - compare R objects x and y testing ‘near equality’ (can set a tolerance) and does not assess attributes

#### Does code produce an output/message/warning/error?
- `expect_error` - tests if an expression throws an error
- `expect_warning` - tests whether an expression outputs a warning
- `expect_output` - tests that print output matches a specified value

#### Is the object true/false?

These are fall-back expectations that you can use when none of the other more specific expectations apply. The disadvantage is that you may get a less informative error message.

- `expect_true` - tests if the object returns `TRUE`
- `expect_false` - tests if the object returns `FALSE`

#### Tolerance and tests: 

Below we add a tolerance arguement to the `expect_equal` statement such that the observed difference between these very similar vectors doesn't cause the test to fail. 

```
x <- c(3.5, 3.5, 3.5)
y <- c(3.5, 3.5, 3.49999)
test_that("x and y should contain the same values", {
    expect_equal(x, y)
})
```

```
Error: Test failed: 'x and y should contain the same values'
* <text>:4: `x` not equal to `y`.
1/3 mismatches
[3] 3.5 - 3.5 == 1e-05
Traceback:

1. test_that("x and y should contain the same values", {
 .     expect_equal(x, y)
 . })
2. test_code(desc, code, env = parent.frame())
3. get_reporter()$end_test(context = get_reporter()$.context, test = test)
4. stop(message, call. = FALSE)
```

In [13]:
x <- c(3.5, 3.5, 3.5)
y <- c(3.5, 3.5, 3.49999)
test_that("x and y should contain the same values", {
    expect_equal(x, y, tolerance = 0.00001)
})

#### Unit test example 

In [14]:
celsius_to_fahr <- function(temp) {
  (temp * (9 / 5)) + 32
}

In [15]:
test_that("Temperature should be the same in Celcius and Fahrenheit at -40", {
        expect_identical(celsius_to_fahr(-40), -40)
    })
test_that("Room temperature should be about 23 degrees in Celcius and 73 degrees Fahrenheit", {
        expect_equal(celsius_to_fahr(23), 73, tolerance = 1)
    })

### Test-driven development (TDD) review

1. Write your tests first (that call the function you haven't yet written), based on edge cases you expect or can calculate by hand

2. If necessary, create some "helper" data to test your function with (this might be done in conjunction with step 1)

3. Write your function to make the tests pass (in this process you might think of more tests that you want to add)

#### Toy example of how TDD can be helpful

Let's create a function called `fahr_to_celsius` that converts temperatures from Fahrenheit to Celsius.

First we'll write the tests (which will fail):


In [16]:
test_fahr_to_celsius <- function() {
    test_that("Temperature should be the same in Celcius and Fahrenheit at -40", {
        expect_identical(fahr_to_celsius(-40), -40)
    })
    test_that("Room temperature should be about 73 degrees Fahrenheit and 23 degrees in Celcius", {
        expect_equal(fahr_to_celsius(73), 23, tolerance = 1)
    })
}

Then we write our function to pass the tests:

In [17]:
fahr_to_celsius <- function(temp) {
    (temp + 32) * 5/9
}

Then we call our tests to check it:

```
test_fahr_to_celsius()
```

```
Error: Test failed: 'Temperature should be the same in Celcius and Fahrenheit at -40'
* <text>:3: fahr_to_celsius(-40) not identical to -40.
1/1 mismatches
[1] -4.44 - -40 == 35.6
Traceback:

1. test_fahr_to_celsius()
2. test_that("Temperature should be the same in Celcius and Fahrenheit at -40", 
 .     {
 .         expect_identical(fahr_to_celsius(-40), -40)
 .     })   # at line 2-4 of file <text>
3. test_code(desc, code, env = parent.frame())
4. get_reporter()$end_test(context = get_reporter()$.context, test = test)
5. stop(message, call. = FALSE)
```

We found an error - so we go back and edit our function:

In [18]:
fahr_to_celsius <- function(temp) {
    (temp - 32) * 5/9
}

And then call our tests again to see if we got it right!

In [19]:
test_fahr_to_celsius()

No message from the test, means we got it this time!

<img src="https://media.giphy.com/media/EXFAJtutz5Ig8/giphy.gif" >

## Exception handling

How to check type and throw an error if not the expected type:

```
if (!is.numeric(c(1, 2, "c")))
  stop("Cannot compute of a vector of characters.")
```

```
Error in eval(expr, envir, enclos): Cannot compute of a vector of characters.
Traceback:

1. stop("Cannot compute of a vector of characters.")
```

Example of defensive programming at the beginning of a function:

In [20]:
fahr_to_celsius <- function(temp) {
    if(!is.numeric(temp)){
        stop("Cannot calculate temperature in Farenheit for non-numerical values")
    }
    (temp - 32) * 5/9
}

```
fahr_to_celsius("thirty")
```

```
Error in fahr_to_celsius("thirty"): Cannot calculate temperature in Farenheit for non-numerical values
Traceback:

1. fahr_to_celsius("thirty")
2. stop("Cannot calculate temperature in Farenheit for non-numerical values")   # at line 3 of file <text>
```

If you wanted to issue a warning instead of an error, you could use `warning` in place of `stop` in the example above. However, in most cases it is better practice to throw an error than to print a warning...

#### We can test our exceptions using test_that:

In [21]:
test_that("Non-numeric values for temp should throw an error", {
    expect_error(fahr_to_celsius("thirty"))
    expect_error(fahr_to_celsius(list(4)))
    })

#### `try` in R

Similar to Python, R has a `try` function to attempt to run code, and continue running subsequent code even if code in the try block does not work:

```
try({
    # some code
    # that can be 
    # split across several
    # lines
})

# code to continue even if error in code 
# in try code block above
```

This code normally results in an error that stops following code from running:

```
x <- data.frame(col1 = c(1, 2, 3, 2, 1), 
                col2 = c(0, 1, 0, 0 , 1))
x[3]
dim(x)
```

```
Error in `[.data.frame`(x, 3): undefined columns selected
Traceback:

1. x[3]
2. `[.data.frame`(x, 3)
3. stop("undefined columns selected")
```

Try let's the code following the error run:

In [22]:
try({x <- data.frame(col1 = c(1, 2, 3, 2, 1), 
                     col2 = c(0, 1, 0, 0 , 1))
     x[3]
})
dim(x)

Error in `[.data.frame`(x, 3) : undefined columns selected


[1] 5 2

Sensibly (IMHO) `try` has a default of `silent=FALSE`, which you can change if you find good reason too.

#### RStudio has template for `roxygen2` documentation

<img src="img/insert_roxygen.png" width=500>

## Reading in functions from an R script

Usually the step before packaging your code, is having some functions in another script that you want to read into your analysis. We use the `source` function to do this:

In [24]:
source("src/kelvin_to_celsius.R")

Once you do this, you have access to all functions contained within that script:

In [25]:
kelvin_to_celsius(273.15)

[1] 0

*Note - this is how the `test_*` functions are brought into your Jupyter notebooks for the autograding part of your lab3 homework.*

## Introduction to R packages

- `source("script_with_functions.R")` is useful, but when you start using these functions in different projects you need to keep copying the script, or having overly specific paths...

- The next step is packaging your R code so that it can be installed and then used across multiple projects on your (and others) machines without directly pointing to where the code is stored, but instead accessed using the `library` function.

- You will learn how to do this in Collaborative Software Development (term 2), but for now, let's tour a simple R package to get a better understanding of what they are: https://github.com/ttimbers/convertemp

### Install the convertemp R package:

In RStudio, type: `devtools::install_github("ttimbers/convertemp")`

In [26]:
library(convertemp)


Attaching package: ‘convertemp’


The following objects are masked _by_ ‘.GlobalEnv’:

    celsius_to_fahr, fahr_to_celsius, kelvin_to_celsius




In [27]:
?celsius_to_kelvin

celsius_to_kelvin {convertemp},R Documentation
temp,numeric


In [28]:
celsius_to_kelvin(0)

[1] 273.15

### Packages and environments

- Each package attached by library() becomes one of the parents of the global environment

- The immediate parent of the global environment is the last package you attached, the parent of that package is the second to last package you attached, …

<img src="https://d33wubrfki0l68.cloudfront.net/038b2da4f5db1d2a8acaf4ee1e7d08d04ab36ebc/ac22a/diagrams/environments/search-path.png" width=800>

*Source: [Advanced R](https://adv-r.hadley.nz/) by Hadley Wickham*

When you attach another package with library(), the parent environment of the global environment changes:

<img src="https://d33wubrfki0l68.cloudfront.net/7c87a5711e92f0269cead3e59fc1e1e45f3667e9/0290f/diagrams/environments/search-path-2.png" width=800>

*Source: [Advanced R](https://adv-r.hadley.nz/) by Hadley Wickham*

### What did we learn today?

- How to write and test functions in R
 
- How to handle exceptions

- How to source functions from other files

- A little bit about what R packages are


## Attribution:
- [Advanced R](https://adv-r.hadley.nz/) by Hadley Wickham
- [The Tidynomicon](http://tidynomicon.tech/) by Greg Wilson